<a href="https://colab.research.google.com/github/jqnatividad/plotly-pages/blob/master/NYC_borough_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Currently, the widely used [JHU COVID19 dashboard](https://coronavirus.jhu.edu/map.html) and even the [NYTimes COVID19 data](https://github.com/nytimes/covid-19-data) only have data at the city level for NYC, not down to the borough level.

Fortunately, [USAFacts just came out with their own COVID19 opendata](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/) with borough level data.  

This notebook wrangles this data and creates two plotly visualizations for confirmed cases and deaths for all 5 boroughs.

First, let's install plotly & pandas...

In [1]:
!pip install plotly
!pip install pandas

In [2]:
import pandas as pd
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv"
confirmed_df=pd.read_csv(url)
confirmed_df

,countyFIPS,County Name,State,stateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10.0
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,3,3,4,4,5,5,10,15,18,19,23.0
3,1005,Barbour County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,1007,Bibb County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3191,56039,Teton County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,3,6,8,12,14,14,17,23,26.0
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1.0
3194,56045,Weston County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


After fetching the data, let's filter it just for NYC boroughs, remove some extraneous columns, and then transpose the data for graphing.

In [3]:
nyc_counties_df = confirmed_df.loc[(confirmed_df['State'] == 'NY') & (confirmed_df['countyFIPS'].isin([36005, 36061, 36081, 36085, 36047]))]

nyc_counties_df=nyc_counties_df.set_index('County Name')
nyc_counties_df=nyc_counties_df.drop(columns=['countyFIPS', 'State', 'stateFIPS'])

nyc_counties_df = nyc_counties_df.transpose()

nyc_counties_df



County Name,Bronx County,Kings County,New York County,Queens County,Richmond County
1/22/2020,0.0,0.0,0.0,0.0,0.0
1/23/2020,0.0,0.0,0.0,0.0,0.0
1/24/2020,0.0,0.0,0.0,0.0,0.0
1/25/2020,0.0,0.0,0.0,0.0,0.0
1/26/2020,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
3/28/2020,5752.0,8129.0,5237.0,9831.0,1781.0
3/29/2020,6250.0,8887.0,5582.0,10737.0,1984.0
3/30/2020,6925.0,10171.0,6060.0,12756.0,2140.0
3/31/2020,7814.0,11160.0,6539.0,13869.0,2354.0


Manhattan ("New York County") had the first COVID19 case, let's filter the dataframe so we start with the first COVID19 confirmed case in Manhattan.

In [4]:
df = nyc_counties_df.reset_index()

df = df.rename(columns={"index":"Date"})
indexNames = df[ df['New York County'] <= 0].index

df.drop(indexNames, inplace=True)

df

County Name,Date,Bronx County,Kings County,New York County,Queens County,Richmond County
40,3/2/2020,0.0,0.0,1.0,0.0,0.0
41,3/3/2020,0.0,0.0,1.0,0.0,0.0
42,3/4/2020,0.0,0.0,1.0,0.0,0.0
43,3/5/2020,0.0,1.0,3.0,0.0,0.0
44,3/6/2020,0.0,1.0,4.0,0.0,0.0
45,3/7/2020,0.0,3.0,8.0,1.0,0.0
46,3/8/2020,1.0,3.0,8.0,1.0,0.0
47,3/9/2020,2.0,4.0,11.0,2.0,1.0
48,3/10/2020,3.0,10.0,17.0,4.0,2.0
49,3/11/2020,3.0,14.0,18.0,4.0,4.0


Create the plotly graph for confirmed cases.

In [16]:
import plotly.graph_objects as go
fig2 = go.Figure()
fig2.add_scatter(x=df['Date'], y=df['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df['Date'], y=df['Kings County'], mode='lines',name='Brooklyn')
fig2.add_scatter(x=df['Date'], y=df['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df['Date'], y=df['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df['Date'], y=df['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df['Date'], 
                 y=df['Richmond County']+df['Bronx County']+df['Kings County']+df['Queens County']+df['New York County'], 
                 mode='lines+markers', name='NYC')
fig2.update_layout(title="NYC COVID19 Confirmed Cases")

In [17]:
from google.colab import drive
drive.mount('/content/drive')

fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19.html')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Connect to Google Drive and save the graph HTML there.

In [18]:
fig2.update_layout(title="NYC COVID19 Confirmed Cases - log scale", yaxis_type="log")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-log.html')

Use the same process for COVID19 deaths...

In [20]:
url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv"
deaths_df = pd.read_csv(url)
deaths_df

,countyFIPS,County Name,State,stateFIPS,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,2/1/2020,2/2/2020,2/3/2020,2/4/2020,2/5/2020,2/6/2020,2/7/2020,2/8/2020,2/9/2020,2/10/2020,2/11/2020,2/12/2020,2/13/2020,2/14/2020,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,2/24/2020,2/25/2020,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020,3/7/2020,3/8/2020,3/9/2020,3/10/2020,3/11/2020,3/12/2020,3/13/2020,3/14/2020,3/15/2020,3/16/2020,3/17/2020,3/18/2020,3/19/2020,3/20/2020,3/21/2020,3/22/2020,3/23/2020,3/24/2020,3/25/2020,3/26/2020,3/27/2020,3/28/2020,3/29/2020,3/30/2020,3/31/2020,4/1/2020
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,1005,Barbour County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,1007,Bibb County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3191,56039,Teton County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3194,56045,Weston County,WY,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [21]:
nyc_counties_deaths_df = deaths_df.loc[(deaths_df['State'] == 'NY') & (deaths_df['countyFIPS'].isin([36005, 36061, 36081, 36085, 36047]))]

nyc_counties_deaths_df=nyc_counties_deaths_df.set_index('County Name')
nyc_counties_deaths_df=nyc_counties_deaths_df.drop(columns=['countyFIPS', 'State', 'stateFIPS'])

nyc_counties_deaths_df = nyc_counties_deaths_df.transpose()

nyc_counties_deaths_df

County Name,Bronx County,Kings County,New York County,Queens County,Richmond County
1/22/2020,0.0,0.0,0.0,0.0,0.0
1/23/2020,0.0,0.0,0.0,0.0,0.0
1/24/2020,0.0,0.0,0.0,0.0,0.0
1/25/2020,0.0,0.0,0.0,0.0,0.0
1/26/2020,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
3/28/2020,152.0,167.0,93.0,216.0,43.0
3/29/2020,188.0,185.0,103.0,253.0,46.0
3/30/2020,215.0,215.0,119.0,305.0,58.0
3/31/2020,262.0,261.0,129.0,376.0,67.0


In [22]:
df = nyc_counties_deaths_df.reset_index()

df = df.rename(columns={"index":"Date"})
indexNames = df[ df['Kings County'] <= 0].index

df.drop(indexNames, inplace=True)

df

County Name,Date,Bronx County,Kings County,New York County,Queens County,Richmond County
52,3/14/2020,0.0,1.0,0.0,0.0,0.0
53,3/15/2020,0.0,1.0,4.0,0.0,0.0
54,3/16/2020,1.0,1.0,0.0,0.0,0.0
55,3/17/2020,1.0,1.0,0.0,0.0,0.0
56,3/18/2020,1.0,1.0,0.0,0.0,0.0
57,3/19/2020,1.0,1.0,0.0,0.0,0.0
58,3/20/2020,1.0,1.0,0.0,0.0,0.0
59,3/21/2020,0.0,1.0,0.0,0.0,0.0
60,3/22/2020,14.0,14.0,10.0,21.0,4.0
61,3/23/2020,28.0,27.0,19.0,40.0,11.0


In [23]:
import plotly.graph_objects as go
fig2 = go.Figure()
fig2.add_scatter(x=df['Date'], y=df['Bronx County'], mode='lines', name='Bronx')
fig2.add_scatter(x=df['Date'], y=df['Kings County'], mode='lines',name='Brooklyn')
fig2.add_scatter(x=df['Date'], y=df['Queens County'], mode='lines', name='Queens')
fig2.add_scatter(x=df['Date'], y=df['New York County'], mode='lines', name='Manhattan')
fig2.add_scatter(x=df['Date'], y=df['Richmond County'], mode='lines', name='Staten Island')
fig2.add_scatter(x=df['Date'], 
                 y=df['Richmond County']+df['Bronx County']+df['Kings County']+df['Queens County']+df['New York County'], 
                 mode='lines+markers', name='NYC')
fig2.update_layout(title="NYC COVID19 Deaths")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-deaths.html')

In [25]:
fig2.update_layout(title="NYC COVID19 Deaths - log scale", yaxis_type="log")

In [0]:
fig2.write_html('/content/drive/My Drive/publicweb/nyc-boroughs-covid19-deaths-log.html')